In [1]:
from itertools import combinations_with_replacement

from ehrhart_quasi_polynomial.ehrhart_piecewise import (
    PiecewiseEhrhartQuasiPolynomial as PEQP,
    create_polytope_from_matrix,
    secondary_fan,
    _process_fan_vectors,
    _compute_change_of_basis_matrices,
    _compute_periods,
    _generate_cone_points)

from ehrhart_quasi_polynomial import *

In [2]:
from sage.calculus.var import var
from sage.functions.other import ceil, factorial
from sage.geometry.cone import Cone
from sage.geometry.polyhedron.constructor import Polyhedron
from sage.matrix.constructor import Matrix
from sage.modules.free_module_element import free_module_element
from sage.modules.free_quadratic_module_integer_symmetric import IntegralLattice
from sage.rings.polynomial.polynomial_ring_constructor import PolynomialRing
from sage.rings.rational_field import QQ
from sage.symbolic.ring import SR

In [3]:
ehr = lambda A, b: ehrhart_quasi_polynomial(create_polytope_from_matrix(A, b).Vrepresentation())
num_int_points = lambda A, b: len(create_polytope_from_matrix(A, b).integral_points())

In [8]:
def test_combinations(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p(b)
            if actual != expected:
                print(b, expected, actual, rounded)
        print("all points were tested")

In [5]:
A = Matrix([[-1, 0], [0, -1], [1, 1]])
p = PEQP(A)

{(0,): (0, 0, 0), (1,): (-1, 0, 0), (2,): (-2, 0, 0)}


In [6]:
p._cone_dicts[0]["polynomials"]

{(0,): 9/2*x^2 + 9/2*x + 1, (1,): 9/2*x^2 + 3/2*x, (2,): 9/2*x^2 - 3/2*x}

In [9]:
test_combinations(p, A)

all points were tested


In [10]:
B = Matrix([[-1, 0], [0, -1], [1, 1], [0, 1]])
q = PEQP(B)

{(0, 0): (0, 0, 0, 0), (0, 1): (0, -1, 1, -2), (0, 2): (0, -2, 2, -4), (0, 3): (0, -3, 3, -6), (0, 4): (0, -4, 4, -8), (1, 0): (-3, 4, -3, 8), (1, 1): (-3, 3, -2, 6), (1, 2): (-3, 2, -1, 4), (1, 3): (-3, 1, 0, 2), (1, 4): (-3, 0, 1, 0), (2, 0): (-6, 8, -6, 16), (2, 1): (-6, 7, -5, 14), (2, 2): (-6, 6, -4, 12), (2, 3): (-6, 5, -3, 10), (2, 4): (-6, 4, -2, 8), (3, 0): (-9, 12, -9, 24), (3, 1): (-9, 11, -8, 22), (3, 2): (-9, 10, -7, 20), (3, 3): (-9, 9, -6, 18), (3, 4): (-9, 8, -5, 16), (4, 0): (-12, 16, -12, 32), (4, 1): (-12, 15, -11, 30), (4, 2): (-12, 14, -10, 28), (4, 3): (-12, 13, -9, 26), (4, 4): (-12, 12, -8, 24)}
{(0, 0): (0, 0, 0, 0), (0, 1): (-1, -1, 0, -1), (0, 2): (-2, -2, 0, -2), (0, 3): (-3, -3, 0, -3), (0, 4): (-4, -4, 0, -4), (1, 0): (1, 4, 1, 4), (1, 1): (0, 3, 1, 3), (1, 2): (-1, 2, 1, 2), (1, 3): (-2, 1, 1, 1), (1, 4): (-3, 0, 1, 0), (2, 0): (2, 8, 2, 8), (2, 1): (1, 7, 2, 7), (2, 2): (0, 6, 2, 6), (2, 3): (-1, 5, 2, 5), (2, 4): (-2, 4, 2, 4), (3, 0): (3, 12, 3, 12), (

In [25]:
q._cone_dicts[1]["lifts"]

{(0, 0): (0, 0, 0, 0),
 (0, 1): (-1, -1, 0, -1),
 (0, 2): (-2, -2, 0, -2),
 (0, 3): (-3, -3, 0, -3),
 (0, 4): (-4, -4, 0, -4),
 (1, 0): (1, 4, 1, 4),
 (1, 1): (0, 3, 1, 3),
 (1, 2): (-1, 2, 1, 2),
 (1, 3): (-2, 1, 1, 1),
 (1, 4): (-3, 0, 1, 0),
 (2, 0): (2, 8, 2, 8),
 (2, 1): (1, 7, 2, 7),
 (2, 2): (0, 6, 2, 6),
 (2, 3): (-1, 5, 2, 5),
 (2, 4): (-2, 4, 2, 4),
 (3, 0): (3, 12, 3, 12),
 (3, 1): (2, 11, 3, 11),
 (3, 2): (1, 10, 3, 10),
 (3, 3): (0, 9, 3, 9),
 (3, 4): (-1, 8, 3, 8),
 (4, 0): (4, 16, 4, 16),
 (4, 1): (3, 15, 4, 15),
 (4, 2): (2, 14, 4, 14),
 (4, 3): (1, 13, 4, 13),
 (4, 4): (0, 12, 4, 12)}

In [12]:
test_combinations(q, B)

all points were tested


In [13]:
C = Matrix([[-1, 0], [0, -1], [1, 2], [0, 1]])
r = PEQP(C)

{(0, 0): (0, 0, 0, 0), (0, 1): (0, 0, 1, -1), (0, 2): (0, 0, 2, -2), (0, 3): (0, 0, 3, -3), (0, 4): (0, 0, 4, -4), (0, 5): (0, 0, 5, -5), (0, 6): (0, 0, 6, -6), (0, 7): (0, 0, 7, -7), (1, 0): (0, 0, -2, 1), (1, 1): (0, 0, -1, 0), (1, 2): (0, 0, 0, -1), (1, 3): (0, 0, 1, -2), (1, 4): (0, 0, 2, -3), (1, 5): (0, 0, 3, -4), (1, 6): (0, 0, 4, -5), (1, 7): (0, 0, 5, -6), (2, 0): (0, 0, -4, 2), (2, 1): (0, 0, -3, 1), (2, 2): (0, 0, -2, 0), (2, 3): (0, 0, -1, -1), (2, 4): (0, 0, 0, -2), (2, 5): (0, 0, 1, -3), (2, 6): (0, 0, 2, -4), (2, 7): (0, 0, 3, -5), (3, 0): (0, 0, -6, 3), (3, 1): (0, 0, -5, 2), (3, 2): (0, 0, -4, 1), (3, 3): (0, 0, -3, 0), (3, 4): (0, 0, -2, -1), (3, 5): (0, 0, -1, -2), (3, 6): (0, 0, 0, -3), (3, 7): (0, 0, 1, -4)}
{(0, 0): (0, 0, 0, 0), (0, 1): (0, 0, 0, -1), (0, 2): (0, 0, 0, -2), (0, 3): (0, 0, 0, -3), (1, 0): (0, -1, -1, 0), (1, 1): (0, -1, -1, -1), (1, 2): (0, -1, -1, -2), (1, 3): (0, -1, -1, -3), (2, 0): (0, -2, -2, 0), (2, 1): (0, -2, -2, -1), (2, 2): (0, -2, -2, -

In [14]:
test_combinations(r, C, range(-6, 7))

all points were tested


In [15]:
#for off_set, poly in list(r._cone_dicts[0]["polynomials"].items()):
#    print(ehr(C, r._rays[2] + off_set.lift()))

In [16]:
D = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
s = PEQP(D)

{(0, 0, 0): (0, 0, 0, 0, 0, 0), (0, 0, 1): (0, 0, 0, 0, 0, -1), (0, 1, 0): (0, 0, 0, 0, -1, 0), (0, 1, 1): (0, 0, 0, 0, -1, -1), (1, 0, 0): (0, 0, 0, -1, 0, 0), (1, 0, 1): (0, 0, 0, -1, 0, -1), (1, 1, 0): (0, 0, 0, -1, -1, 0), (1, 1, 1): (0, 0, 0, -1, -1, -1)}


In [17]:
s._cone_dicts[0]["polynomials"]

{(0, 0, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 4*x1*x2 + 2*x0 + 2*x1 + 2*x2 + 1,
 (0, 0, 1): 8*x0*x1*x2 + 4*x0*x1 + 4*x0*x2 + 2*x0,
 (0, 1, 0): 8*x0*x1*x2 + 4*x0*x1 + 4*x1*x2 + 2*x1,
 (0, 1, 1): 8*x0*x1*x2 + 4*x0*x1,
 (1, 0, 0): 8*x0*x1*x2 + 4*x0*x2 + 4*x1*x2 + 2*x2,
 (1, 0, 1): 8*x0*x1*x2 + 4*x0*x2,
 (1, 1, 0): 8*x0*x1*x2 + 4*x1*x2,
 (1, 1, 1): 8*x0*x1*x2}

In [18]:
test_combinations(s, D)

all points were tested


In [19]:
E = Matrix([[-1, 0, 0], [0, -1, 0], [0, 0, -1], [1, 1, 1]])
t = PEQP(E)

{(0,): (0, 0, 0, 0), (1,): (-1, -1, 0, 1), (2,): (-2, -2, 0, 2), (3,): (-3, -3, 0, 3)}


In [20]:
t._cone_dicts[0]["polynomials"]

{(0,): 32/3*x^3 + 16*x^2 + 22/3*x + 1,
 (1,): 32/3*x^3 + 8*x^2 + 4/3*x,
 (2,): 32/3*x^3 - 2/3*x,
 (3,): 32/3*x^3 - 8*x^2 + 4/3*x}

In [21]:
test_combinations(t, E)

all points were tested


In [22]:
ehr(E, t._rays[0])

QuasiPolynomialElement(Ring of Quasi-Polynomials over Rational Field, [[1], [22/3], [16], [32/3]])

In [23]:
F = Matrix([[-1, 0], [0, -1], [1, 3], [0, 1]])
u = PEQP(F)

{(0, 0): (0, 0, 0, 0), (0, 1): (13, -7, 13, -47), (0, 2): (26, -14, 26, -94), (0, 3): (39, -21, 39, -141), (0, 4): (52, -28, 52, -188), (0, 5): (65, -35, 65, -235), (0, 6): (78, -42, 78, -282), (0, 7): (91, -49, 91, -329), (0, 8): (104, -56, 104, -376), (0, 9): (117, -63, 117, -423), (0, 10): (130, -70, 130, -470), (0, 11): (143, -77, 143, -517), (0, 12): (156, -84, 156, -564), (0, 13): (169, -91, 169, -611), (0, 14): (182, -98, 182, -658), (0, 15): (195, -105, 195, -705), (0, 16): (208, -112, 208, -752), (0, 17): (221, -119, 221, -799), (0, 18): (234, -126, 234, -846), (0, 19): (247, -133, 247, -893), (0, 20): (260, -140, 260, -940), (0, 21): (273, -147, 273, -987), (0, 22): (286, -154, 286, -1034), (0, 23): (299, -161, 299, -1081), (0, 24): (312, -168, 312, -1128), (0, 25): (325, -175, 325, -1175), (0, 26): (338, -182, 338, -1222), (0, 27): (351, -189, 351, -1269), (0, 28): (364, -196, 364, -1316), (0, 29): (377, -203, 377, -1363), (0, 30): (390, -210, 390, -1410), (0, 31): (403, -21

KeyboardInterrupt: 

In [ ]:
test_combinations(u, F)

## trying to figure out what the right coordinate transformation is

In [ ]:
p._cone_dicts[0]["polynomials"]

In [ ]:
S = PolynomialRing(QQ, "y", 3)

In [ ]:
t = p._cone_dicts[0]["change_of_basis_matrix"]*free_module_element(p._cone_dicts[0]["scaled_rays"][0])*free_module_element(S.gens())
t

In [ ]:
dict = p._cone_dicts[0]
dict["transformed"] = {}
for off_set, poly in dict["polynomials"].items():
    y = dict["change_of_basis_matrix"]*(free_module_element(S.gens(), SR) - free_module_element(off_set.lift(), SR))
    dict["transformed"][off_set] = dict["polynomials"][off_set](y[0])
dict["transformed"]

In [ ]:
dict["transformed"][list(dict["transformed"].keys())[0]](y0=2, y1=2, y2=1)

In [ ]:
num_int_points(A, (2, 2, 1))

In [ ]:
p((2, 2, 1))[0]

In [ ]:
def test_transform(self, point):
    if len(point) != self._amb_dim:
            raise ValueError("Dimension of ``point`` needs to be equal to the ambient"
                              f" dimension of ``self`` which is {self._amb_dim}.")
    
    for k, cone_dict in enumerate(self._cone_dicts):
            if point in cone_dict["cone"]:
                # print(True)
                off_set = cone_dict["quotient"][cone_dict["quotient"](point)[0]]
                return dict["transformed"][off_set](y0=point[0], y1=point[1], y2=point[2])
    return 0

p.test = test_transform

In [ ]:
def test(p, A, xrange=range(-10, 11)):
        for b in combinations_with_replacement(xrange, A.nrows()):
            expected = num_int_points(A, b)
            actual = p.test(p, b)
            if actual != expected:
                print(b, expected, actual)
        print("all points were tested")

In [ ]:
test(p, A, xrange=range(-5, 6))